*Привет!*

*Меня зовут Фёдор, я буду проводить ревью твоего проекта.*

*Предлагаю общаться на «ты» :)*

*Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать отличным специалистом. Ниже ты найдешь мои комментарии, оформленные вот так: «Комментарий ревьюера:…»*

*Помни, что все мои комментарии являются рекомендациями, а не требованиями – только ты решаешь, как сделать этот проект лучше.*

<div class="alert alert-danger">
<b>Комментарий ревьюера ❌:</b> </div>
    
Так выделены самые важные замечания. Их нужно обязательно отработать, перед тем как показывать работу другим специалистам по Data Science.
    

<div class="alert alert-warning">
<b>Комментарий ревьюера ⚡:</b> </div>
    
Так выделены небольшие замечания. Я надеюсь, что их ты тоже учтешь - твой проект от этого станет только лучше.
    
<div class="alert alert-success">
<b>Комментарий ревьюера ✅:</b> </div>
    
Так я выделяю правильные и верные действия, а так же все остальные комментарии.

<div class="alert alert-success">
<b>Комментарии к приложению✅:</b> 

- requirements - проверь, пожалуйста, не попало ли туда чего-то лишнего. подозреваю, что, например, виджеты или jupyter не являются необходимыми компонентами именно для сборки приложения
- работающее приложение - супер-круто, молодец! Но обрати, пожалуйста, внимание: я столкнулся с тем, что при вводе нового (пользовательского) запроса возвращается тот же результат, что и к дефолтному (про месячные). Как будто бы какая-то переменная не обновляется.
    
- **translate.py** Классный подход с переводом. Еще можно было бы не использовать LLM, а обращаться к тому же google translate, например, так
```python
    from deep_translator import GoogleTranslator
    translator = GoogleTranslator(source='en', target='ru')
    v = 'some text'
    tv = translator.translate(v)
```
- **dataload.py** не смотря на то, что датасет открытый, хранить его в репозитории все рано не стоит. А при except рекомендуется указывать [тип ожидаемой ошибки](https://www.w3schools.com/python/python_try_except.asp), потому что разные ошибки могут требовать разного поведения приложения
- **prediction.py**. строка 15. Кажется, хардкодить N - не лучшая идея

- **dataload.py**. Если скачиваем датасет - лучше это делать через requests, скачивать файл во временную папку, а затем оттуда читать. Это позволит обрабатывать ошибки при обращении к веб-странице. А еще безопаснее было бы давать путь к файлу 
    ```python
    os.path.join(os.dirname(__file__), DATA_PATH)
    ```
Дело в том, что написанный тобой вариант не сработает, если изменится точка запуска скрипта dataload.py. То есть, если находится в той же папке, что и скрипт и запускать его через 
    ```bash
    python3 dataload.py
    ```
    все будет хорошо, а вот если запускать откуда-то еще - 
    ```bash
    python3 some_dir/dataload.py
    ```
    путь к файлу потеряется

    
А теперь посмотрим на исследовательскую часть

</div>
    

In [1]:
import os
import re
import string
import itertools
import unicodedata

import pandas as pd
import numpy as np

from scipy.spatial.distance import cdist
#from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler

from datasets import load_dataset

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer, util

from sklearn.feature_extraction.text import TfidfVectorizer

import warnings

pd.set_option('display.max_rows', None)
pd.set_option("display.max_colwidth", 500)
pd.options.mode.chained_assignment = None
warnings.filterwarnings(action="ignore", message="Mean of empty slice")
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /home/sumkin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /home/sumkin/nltk_data...
[nltk_data] Downloading package punkt to /home/sumkin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sumkin/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /home/sumkin/nltk_data...


ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
def get_info_on_dataset(_df):
    CONST_IN_PERCENTAGES = float(100.0)
    try:
        display(_df.head())
        print("")
        _df.info()
        df_review_info = pd.DataFrame({'Not_Valid_Values_in %': (_df.isna().sum()/len(_df))*CONST_IN_PERCENTAGES} )
        print("")
        display(df_review_info.sort_values(by='Not_Valid_Values_in %', ascending=False))
        print("")
        print("Number of duplicated rows in the dataset: {}".format(_df.duplicated().sum()))
        print("")
        print("Number of unique values in the dataset:\n{}".format(_df.nunique()))
    except:
        print("Can't provide review on given dataset")

def adjust_total_stop_words_list(_stop_words_list):
    total_stop_words_sample = ' '.join(_stop_words_list)
    total_stop_words_sample_processed = unicodedata.normalize('NFKD',
                                                              total_stop_words_sample).encode('ascii','ignore').decode()
    total_stop_words_sample_processed = punctuation_and_case(total_stop_words_sample_processed)
    total_stop_words_list_processed = total_stop_words_sample_processed.split(" ")
    return total_stop_words_list_processed

def punctuation_and_case(text_sample):
    string_punctuation = string.punctuation.replace('-','') + "’"
    punctuation_map = str.maketrans(string_punctuation, ' '*len(string_punctuation))
    return text_sample.translate(punctuation_map)

In [ ]:
STOP_WORDS_LANGUAGES_LIST_CONST = ['english'] 
total_stop_words_lists = [stopwords.words(_lng) for _lng in STOP_WORDS_LANGUAGES_LIST_CONST]
total_stop_words_list = list(itertools.chain(*total_stop_words_lists))
TOTAL_STOP_WORDS_LIST_PROCESSED_CONST = adjust_total_stop_words_list(total_stop_words_list)
TF_IDF_MIN_DF_VALUE_CONST = int(3)
TF_IDF_MAX_DF_VALUE_CONST = float(0.5)
MAX_LENGTH = 100
NUM_BEAMS = 3
EARLY_STOP = True

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
model_translate = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru")

model_translate_glaz = "glazzova/ml_translation_model1"
translator = pipeline("translation", model=model_translate_glaz)

In [ ]:
def lowering(text_sample):
    processed_words_list = [w.lower() for w in text_sample.split()]
    return ' '.join(processed_words_list)

def stop_words(text_sample, _stop_words_ = TOTAL_STOP_WORDS_LIST_PROCESSED_CONST):
    text_sample_processed = [word for word in text_sample.split() if word not in _stop_words_]
    return ' '.join(text_sample_processed)

def strip_hashtags(text_sample):
    processed_words_list = []
    for word in text_sample.split(' '):
        word = word.strip()
        if word:
            if word.find('@') < 0 and word.find('#') < 0 :
                processed_words_list.append(word)
    return ' '.join(processed_words_list)

def alphanumeric_adjustment(text_sample):
    def if_word_valid(w):
        if w.isalnum() and not w.isalpha() and not w.isdigit():
            return False
        else:
            return True
    processed_words_list = [word for word in text_sample.split() if if_word_valid(word)]
    return ' '.join(processed_words_list)

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemm_sample(text_sample):
    wnl = nltk.stem.WordNetLemmatizer()
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(text_sample))
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    text_sample_processed = []
    for word, tag in wordnet_tagged:
        if tag is None:
            text_sample_processed.append(word)
        else:
            text_sample_processed.append(wnl.lemmatize(word, tag))
    return ' '.join(text_sample_processed)

def text_sample_lem_processing(text_sample, _stop_words_remove=False):
    text_sample_processed = unicodedata.normalize('NFKD', text_sample).encode('ascii','ignore').decode()
    text_sample_processed = strip_hashtags(text_sample_processed)
    text_sample_processed = punctuation_and_case(text_sample_processed)
    text_sample_processed = lowering(text_sample_processed)
    if _stop_words_remove:
        text_sample_processed = stop_words(text_sample_processed)
    text_sample_processed = lemm_sample(text_sample_processed)
    text_sample_processed = alphanumeric_adjustment(text_sample_processed)
    return text_sample_processed

def translate_text(text):
    sentences = sent_tokenize(text)
    translated_sentences = []
    for sentence in sentences:
        inputs = tokenizer.encode(sentence, return_tensors="pt")
        outputs = model_translate.generate(inputs, max_length=MAX_LENGTH, num_beams=NUM_BEAMS, early_stopping=EARLY_STOP)
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        translated_sentences.append(translated_text)
    translated_text = " ".join(translated_sentences)
    return translated_text

def translate_text_glaz(text):
    return translator(text)[0]['translation_text']

def calculate_cosine_similarity(v1, v2):
    emb1 = model.encode(v1, convert_to_tensor=True)
    emb2 = model.encode(v2, convert_to_tensor=True)
    cos_sim = util.cos_sim(emb1, emb2)
    return cos_sim.tolist()[0][0]

<div class="alert alert-warning">
<b>Тут лучше было бы делать функции более универсальными, чтобы можно было в качестве параметра передавать модель (разные модели для перевода) или разные токенайзеры ⚡:</b> </div>
    



In [ ]:
DATA_DIR = "data"
os.makedirs(f'../{DATA_DIR}', exist_ok=True)

dataset = load_dataset("medical_questions_pairs")
train_df = dataset["train"].to_pandas()
train_df.to_csv(f'../{DATA_DIR}/train_df.csv')

In [ ]:
get_info_on_dataset(train_df)

In [ ]:
train_df['label'].value_counts()#normalize=True)

In [ ]:
try:
    train_df = pd.read_csv(f'../{DATA_DIR}/train_df_processed.csv', index_col=[0])

except:
    train_df['question_1_processed'] = train_df['question_1'].map(lambda _sample: text_sample_lem_processing(_sample, _stop_words_remove=True))
    train_df['question_2_processed'] = train_df['question_2'].map(lambda _sample: text_sample_lem_processing(_sample, _stop_words_remove=True))
    train_df['len_question_1'] = train_df['question_1'].map(lambda x: len(x)) 
    train_df['len_question_2'] = train_df['question_2'].map(lambda x: len(x)) 
    train_df['text_len_question_1'] = train_df['question_1_processed'].map(lambda x: len(x)) 
    train_df['text_len_question_2'] = train_df['question_2_processed'].map(lambda x: len(x)) 
    train_df['rus_question_1'] = train_df['question_1'].apply(translate_text)
    train_df['rus_question_2'] = train_df['question_2'].apply(translate_text)
    train_df['rusgl_question_1'] = train_df['question_1'].apply(translate_text_glaz)
    train_df['rusgl_question_2'] = train_df['question_2'].apply(translate_text_glaz)

<div class="alert alert-warning">
<b>Вот тут не хватает пояснений, что делает эта конструкция. Если не получилось прочитать файл по заданному пути - то - что? ⚡:</b> </div>
    

    



In [ ]:
get_info_on_dataset(train_df)

In [ ]:
train_df[['question_1_processed','question_1']].head()

In [ ]:
train_df[['question_2_processed','question_2']].head()

In [ ]:
print(len(train_df[train_df['len_question_1'] <= 0]))
print(len(train_df[train_df['len_question_2'] <= 0]))
print(len(train_df[train_df['text_len_question_1'] <= 0]))
print(len(train_df[train_df['text_len_question_2'] <= 0]))

<div class="alert alert-warning">
<b>Кажется, проверка на отрицательную длину предложения будет всегда давать один и тот же результат. А вот отбросить слишком длинные или слишком короткие предлодения - можно было бы ⚡:</b> </div>



In [ ]:
train_df.describe()

In [ ]:
column_to_translate = ['question_1','question_2']
translated_columns = ['rus_question_1','rus_question_2']
train_df[translated_columns+column_to_translate+['label']].head()


    

    
<div class="alert alert-success">
<b>Классный подход!✅:</b> </div>


In [ ]:
corpus = pd.concat([train_df['question_1_processed'],train_df['question_2_processed']]).drop_duplicates(keep='first')
corpus.head(), corpus.shape

In [ ]:
corpus_full = pd.concat([train_df['question_1'],train_df['question_2']]).drop_duplicates(keep='first')
corpus_full.head(), corpus_full.shape

In [ ]:
tf_idf_vectorizer = TfidfVectorizer()

In [ ]:
tf_idf_corpus = tf_idf_vectorizer.fit_transform(corpus)
tf_idf_q1 = tf_idf_vectorizer.transform(train_df['question_1_processed'])
tf_idf_q2 = tf_idf_vectorizer.transform(train_df['question_2_processed'])
tf_idf_q1_array = tf_idf_q1.toarray()
tf_idf_q2_array = tf_idf_q2.toarray()
tf_idf_corpus_array = tf_idf_corpus.toarray()

In [ ]:
tf_idf_corpus_full = tf_idf_vectorizer.fit_transform(corpus_full)
tf_idf_q1_full = tf_idf_vectorizer.transform(train_df['question_1'])
tf_idf_q2_full = tf_idf_vectorizer.transform(train_df['question_2'])
tf_idf_q1_full_array = tf_idf_q1_full.toarray()
tf_idf_q2_full_array = tf_idf_q2_full.toarray()
tf_idf_corpus_full_array = tf_idf_corpus_full.toarray()

In [ ]:
#tf_idf_vectorizer.vocabulary_

In [ ]:
tf_idf_corpus_array.shape

In [ ]:
tf_idf_q1_ = pd.Series(tf_idf_q1_array.tolist())
tf_idf_q2_ = pd.Series(tf_idf_q2_array.tolist())
train_df['tf_idf_q1'] = tf_idf_q1_
train_df['tf_idf_q2'] = tf_idf_q2_
train_df['cdist_cb'] = [cdist([row['tf_idf_q1']], [row['tf_idf_q2']],'cityblock')[0][0] for _, row in train_df.iterrows()]
train_df['cdist_eu'] = [cdist([row['tf_idf_q1']], [row['tf_idf_q2']],'euclidean')[0][0] for _, row in train_df.iterrows()]

tf_idf_q1_full_ = pd.Series(tf_idf_q1_full_array.tolist())
tf_idf_q2_full_ = pd.Series(tf_idf_q2_full_array.tolist())
train_df['tf_idf_q1_full'] = tf_idf_q1_full_
train_df['tf_idf_q2_full'] = tf_idf_q2_full_
train_df['cdist_cb_full'] = [cdist([row['tf_idf_q1_full']], [row['tf_idf_q2_full']],'cityblock')[0][0] for _, row in train_df.iterrows()]
train_df['cdist_eu_full'] = [cdist([row['tf_idf_q1_full']], [row['tf_idf_q2_full']],'euclidean')[0][0] for _, row in train_df.iterrows()]

In [ ]:
train_df[['question_1','question_2','label','cdist_cb','cdist_eu','cdist_cb_full','cdist_eu_full']].head()#.query('question_1 == @s')

In [ ]:
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

embeddings1 = model.encode(train_df['question_1'].tolist())
embeddings2 = model.encode(train_df['question_2'].tolist())
train_df['question_1_mLM'] = embeddings1.tolist()
train_df['question_2_mLM'] = embeddings2.tolist()
train_df['cos_sim_mLM'] = train_df.apply(lambda x: calculate_cosine_similarity(x['question_1'], x['question_2']), axis=1)

In [ ]:
train_df[['question_1','question_2','label','cdist_cb','cdist_eu','cos_sim_mLM']].query('label == 1').sort_values(by='cos_sim_mLM', ascending=False).tail(5)

In [ ]:
train_df[['question_1','question_2','label','cdist_cb','cdist_eu','cos_sim_mLM']].query('label == 1').sort_values(by='cos_sim_mLM', ascending=False).head(5)

In [ ]:
train_df.describe()

In [ ]:
scaler_cols = ['cdist_cb','cdist_eu','cdist_cb_full','cdist_eu_full']
scaled_cols = ['cdist_cb_scl','cdist_eu_scl','cdist_cb_full_scl','cdist_eu_full_scl']

scaler = MinMaxScaler()

scaled_data = pd.DataFrame(scaler.fit_transform(train_df[scaler_cols]), columns=scaled_cols)
scaled_data.describe()

In [ ]:
train_df = pd.concat([train_df,scaled_data],axis=1)
train_df.info()

In [ ]:
train_df.query('label == 0').describe()

In [ ]:
train_df.query('label == 1').describe()

In [ ]:
def accuracy_topN(question_1, question_2, def_predict, metric_=None, N=5):

    corr_pred = 0
    all_pred = 0
    
    question_1_array = np.vstack(question_1)
    question_2_array = np.vstack(question_2)

    if def_predict=='cdist_':
        predict_1 = np.array([cdist(question_2_array, np.expand_dims(q1, axis=0), metric=metric_).reshape(-1) for q1 in question_1_array])
        predict_2 = np.array([cdist(question_1_array, np.expand_dims(q2, axis=0), metric=metric_).reshape(-1) for q2 in question_2_array])
    elif def_predict=='cos_sim_':
        predict_1 = np.array([util.cos_sim(question_2_array, q1).reshape(-1) for q1 in question_1_array])
        predict_2 = np.array([util.cos_sim(question_1_array, q2).reshape(-1) for q2 in question_2_array])
        
    for i in range(len(question_1_array)):
        values_top = np.argsort(predict_1[i])[::-1][:N]  
        if i in values_top:
            corr_pred += 1
        all_pred += 1
    
    for i in range(len(question_2_array)):
        values_top = np.argsort(predict_2[i])[::-1][:N]
        if i in values_top:
            corr_pred += 1
        all_pred += 1
    
    accuracy_top = corr_pred / all_pred
    return accuracy_top


In [ ]:
accuracy_top5_cos_sim = accuracy_topN(train_df['question_1_mLM'], train_df['question_2_mLM'], def_predict='cos_sim_', N=5)
accuracy_top10_cos_sim = accuracy_topN(train_df['question_1_mLM'], train_df['question_2_mLM'], def_predict='cos_sim_', N=10)
accuracy_top5_cos_sim, accuracy_top10_cos_sim

In [ ]:
accuracy_top5_cos_sim_tf = accuracy_topN(train_df['tf_idf_q1'], train_df['tf_idf_q2'], def_predict='cos_sim_', N=5)
accuracy_top10_cos_sim_tf = accuracy_topN(train_df['tf_idf_q1'], train_df['tf_idf_q2'], def_predict='cos_sim_', N=10)
accuracy_top5_cos_sim_tf, accuracy_top10_cos_sim_tf


    

    
<div class="alert alert-success">
<b>Вот тут все хорошо, и метрика довольно высокая ✅:</b> </div>


In [ ]:
accuracy_top5_cdist_cb = accuracy_topN(train_df['question_1_mLM'], train_df['question_2_mLM'], def_predict='cdist_', metric_='cityblock', N=5)
accuracy_top10_cdist_cb = accuracy_topN(train_df['question_1_mLM'], train_df['question_2_mLM'], def_predict='cdist_', metric_='cityblock', N=10)
accuracy_top5_cdist_cb, accuracy_top10_cdist_cb

In [ ]:
accuracy_top5_cdist_eu = accuracy_topN(train_df['question_1_mLM'], train_df['question_2_mLM'], def_predict='cdist_', metric_='euclidean', N=5)
accuracy_top10_cdist_eu = accuracy_topN(train_df['question_1_mLM'], train_df['question_2_mLM'], def_predict='cdist_', metric_='euclidean', N=10)
accuracy_top5_cdist_eu, accuracy_top10_cdist_eu

<div class="alert alert-warning">
<b>А вот тут нужны пояснения, как трактовать такие значения accuracy ⚡:</b> </div>
    



In [ ]:
# accuracy_top5_cdist_cb_tf = accuracy_topN(train_df['tf_idf_q1'], train_df['tf_idf_q2'], def_predict='cdist_', metric_='cityblock', N=5)
# accuracy_top10_cdist_cb_tf = accuracy_topN(train_df['tf_idf_q1'], train_df['tf_idf_q2'], def_predict='cdist_', metric_='cityblock', N=10)
# accuracy_top5_cdist_cb_tf, accuracy_top10_cdist_cb_tf

# accuracy_top5_cdist_eu_tf = accuracy_topN(train_df['tf_idf_q1'], train_df['tf_idf_q2'], def_predict='cdist_', metric_='euclidean', N=5)
# accuracy_top10_cdist_eu_tf = accuracy_topN(train_df['tf_idf_q1'], train_df['tf_idf_q2'], def_predict='cdist_', metric_='euclidean', N=10)
# accuracy_top5_cdist_eu_tf, accuracy_top10_cdist_eu_tf

# accuracy_top5_cos_sim_tf_full = accuracy_topN(train_df['tf_idf_q1_full'], train_df['tf_idf_q2_full'], def_predict='cos_sim_', N=5)
# accuracy_top10_cos_sim_tf_full = accuracy_topN(train_df['tf_idf_q1_full'], train_df['tf_idf_q2_full'], def_predict='cos_sim_', N=10)
# accuracy_top5_cos_sim_tf_full, accuracy_top10_cos_sim_tf_full

# accuracy_top5_cdist_cb_tf_full = accuracy_topN(train_df['tf_idf_q1_full'], train_df['tf_idf_q2_full'], def_predict='cdist_', metric_='cityblock', N=5)
# accuracy_top10_cdist_cb_tf_full = accuracy_topN(train_df['tf_idf_q1_full'], train_df['tf_idf_q2_full'], def_predict='cdist_', metric_='cityblock', N=10)
# accuracy_top5_cdist_cb_tf_full, accuracy_top10_cdist_cb_tf_full

# accuracy_top5_cdist_eu_tf_full = accuracy_topN(train_df['tf_idf_q1_full'], train_df['tf_idf_q2_full'], def_predict='cdist_', metric_='euclidean', N=5)
# accuracy_top10_cdist_eu_tf_full = accuracy_topN(train_df['tf_idf_q1_full'], train_df['tf_idf_q2_full'], def_predict='cdist_', metric_='euclidean', N=10)
# accuracy_top5_cdist_eu_tf_full, accuracy_top10_cdist_eu_tf_full

In [ ]:
questions = pd.concat([train_df['question_1'],train_df['question_2']],axis=0).drop_duplicates(keep='first')
questions_embeddings = model.encode(questions.tolist())

In [ ]:
q = questions.sample().values[0]

inputs = tokenizer.encode(q, return_tensors="pt")
outputs = model_translate.generate(inputs, max_length=1000, num_beams=5, early_stopping=False)
translated_question = tokenizer.decode(outputs[0], skip_special_tokens=True)

question_embedding = model.encode(q)
predict = np.array([util.cos_sim(questions_embeddings, question_embedding)])

N = 11
values_top = np.argsort(predict.reshape(-1))[::-1][:N+1]

df_ = pd.DataFrame(columns=['questions', 'translated_questions','cos_sim']) 
for val in values_top:
    if questions.tolist()[val] == q:
        pass        
    else: 
        q_ = questions.tolist()[val]
        cos_sim_ = predict.reshape(-1)[val].round(1)
        translated_q_ = translate_text(q_)
        df_.loc[val] = [q_,translated_q_,cos_sim_]

print(q, translated_question)
display(df_[['questions', 'translated_questions','cos_sim']])


    

    
<div class="alert alert-success">
<b>Это прекрасно!!))))  ✅:</b>
    
    
    
[Напоминает бессмертную классику](http://lib.ru/ANEKDOTY/mouse_driver.txt)
</div>


In [ ]:
train_df.query('(question_1 == @q|question_2 == @q)')[['question_1','question_2','label','cos_sim_mLM']]

In [ ]:
train_df.to_csv(f'../{DATA_DIR}/train_df_processed.csv')
train_df = pd.read_csv(f'../{DATA_DIR}/train_df_processed.csv', index_col=[0])

In [ ]:
train_df.info()


    

    
<div class="alert alert-success">
<b>Вывод ревьюера:</b>
    
У тебя получилась отличная работа. Проведено интересное исследование, есть готовое работающее приложение. 
    
Мне очень понравилось!
    
Удачи в дальнейших проектах!

</div>
